In [5]:
!git init

Reinitialized existing Git repository in /content/.git/


In [6]:
!git checkout -b main

Switched to a new branch 'main'


In [7]:
!git remote add origin https://github.com/dinhoitt/AI_Apply_final.git

In [8]:
!git remote -v

origin	https://github.com/dinhoitt/AI_Apply_final.git (fetch)
origin	https://github.com/dinhoitt/AI_Apply_final.git (push)


In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 기말고사

by 2021706161 박태수

# 1 L19

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
import random

# Load the Harry Potter dataset
dataset = load_dataset("KaungHtetCho/Harry_Potter_LSTM")

random_sentence = random.choice(dataset['train']['text'])
print(f"Random Sentence: {random_sentence[:100]}...")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)in/Harry Potter 1 - Sorcerer's Stone.txt:   0%|          | 0.00/443k [00:00<?, ?B/s]

(…)/Harry Potter 2 - Chamber of Secrets.txt:   0%|          | 0.00/490k [00:00<?, ?B/s]

(…)y Potter 3 - The Prisoner Of Azkaban.txt:   0%|          | 0.00/615k [00:00<?, ?B/s]

...

(…)est/Harry Potter 7 - Deathly Hollows.txt:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/57435 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5897 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6589 [00:00<?, ? examples/s]

Random Sentence: anger? He could as easily detach his legs......


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
import random


# 재현성을 위한 시드 설정
torch.manual_seed(100)

# 문자 사전(Dictionary) 생성
char_set = list(set(random_sentence))  # 랜덤 문장에서 문자 집합 생성
dic = {c: i for i, c in enumerate(char_set)}  # 각 문자에 인덱스 할당

# 파라미터 설정
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# 입력 및 타겟 배치 준비
input_batch = []
target_batch = []

for i in range(0, len(random_sentence) - unit_sequence_length):
    input_sequence = random_sentence[i:i + unit_sequence_length]
    target_sequence = random_sentence[i + 1:i + unit_sequence_length + 1]
    input_batch.append([dic[char] for char in input_sequence])
    target_batch.append([dic[char] for char in target_sequence])

# 입력 및 타겟 배치 크기 확인
print(f"Input Batch Size: {len(input_batch)} x {len(input_batch[0])}")
print(f"Target Batch Size: {len(target_batch)} x {len(target_batch[0])}")

batch_size = len(input_batch)
sequence_length = len(input_batch[0])
input_size = len(dic)

# 원-핫 인코딩된 입력 생성
X = torch.zeros(batch_size, sequence_length, input_size)
for i, seq in enumerate(input_batch):
    for j, char_index in enumerate(seq):
        X[i, j, char_index] = 1

# 타겟을 LongTensor로 변환
Y = torch.LongTensor(target_batch)  # (batch_size, sequence_length)

# 학습 파라미터 설정
learning_rate = 0.05
training_epochs = 100
criterion = nn.CrossEntropyLoss()

# LSTM 기반 RNN 정의
class Custom_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(Custom_RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_state):
        h, c = hidden_state
        out, (h, c) = self.rnn(x, (h, c))
        hidden_state = (h, c)
        out = self.fc(out)
        return out, hidden_state

# 모델, 옵티마이저, 손실 함수 초기화
model = Custom_RNN(input_size, hidden_size, output_size, num_layers=1)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
print("\nTraining with LSTM...")
hidden_state = None  # 은닉 상태 초기화
for epoch in range(training_epochs):
    optimizer.zero_grad()

    # 순전파
    if hidden_state is None:
        hidden_state = (
            torch.zeros(1, X.size(0), hidden_size),
            torch.zeros(1, X.size(0), hidden_size),
        )
    else:
        hidden_state = (hidden_state[0].detach(), hidden_state[1].detach())

    outputs, hidden_state = model(X, hidden_state)

    # 역전파 및 최적화
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    if epoch % 10 == 9:
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

# 텍스트 생성
print("\nGenerating text...")
with torch.no_grad():
    input_char = random_sentence[:unit_sequence_length]
    input_data = [dic[char] for char in input_char]
    input_tensor = torch.zeros(1, unit_sequence_length, dic_size)
    for i, char_index in enumerate(input_data):
        input_tensor[0, i, char_index] = 1

    result = input_char
    hidden_state = None
    for _ in range(len(random_sentence) - unit_sequence_length):
        if hidden_state is None:
            hidden_state = (
                torch.zeros(1, 1, hidden_size),
                torch.zeros(1, 1, hidden_size),
            )

        outputs, hidden_state = model(input_tensor, hidden_state)
        pred_char_index = outputs[0, -1, :].argmax().item()
        result += char_set[pred_char_index]

        # input_tensor 업데이트
        input_tensor = torch.zeros(1, unit_sequence_length, dic_size)
        for i, char in enumerate(result[-unit_sequence_length:]):
            input_tensor[0, i, dic[char]] = 1

    print(f"Original: {random_sentence}")
    print(f"Generated: {result}")

    # 정확도 계산
    correct = sum([1 for i in range(len(result)) if result[i] == random_sentence[i]])
    accuracy = correct / len(result) * 100
    print(f"Accuracy: {accuracy:.2f}%")


Input Batch Size: 24 x 20
Target Batch Size: 24 x 20

Training with LSTM...
Epoch: 10, Loss: 1.7798705101013184
Epoch: 20, Loss: 0.3415728211402893
Epoch: 30, Loss: 0.06163140758872032
Epoch: 40, Loss: 0.028139939531683922
Epoch: 50, Loss: 0.021267293021082878
Epoch: 60, Loss: 0.0249104555696249
Epoch: 70, Loss: 0.021673839539289474
Epoch: 80, Loss: 0.020067444071173668
Epoch: 90, Loss: 0.019261496141552925
Epoch: 100, Loss: 0.019274761900305748

Generating text...
Original: anger? He could as easily detach his legs...
Generated: anger? He could as easily detach his legs...
Accuracy: 100.00%


# 2 L20

In [9]:
!pip install transformers datasets


In [12]:
from transformers import pipeline
import random
import datasets
import numpy as np

# 1. 한국어 감정 분류를 위한 Huggingface 모델 2개 선택
model_names = [
    "beomi/KcELECTRA-base",  # 첫 번째 모델
    "monologg/koelectra-base-v3-discriminator"  # 두 번째 모델
]

# 선택한 모델 로드
models = [pipeline("text-classification", model=model) for model in model_names]

# 2. 데이터셋 로드
dataset = datasets.load_dataset("sepidmnorozy/Korean_sentiment")

# 테스트 데이터 추출
test_data = dataset["train"]

# 3. 랜덤하게 100개의 샘플을 선택하여 평가에 활용
random.seed(42)  # 재현성을 위한 시드 설정
sample_data = random.sample(list(test_data), 100)  # 100개의 샘플 추출
texts = [item["text"] for item in sample_data]  # 문장 텍스트 추출
labels = [item["label"] for item in sample_data]  # 정답 라벨 추출

# 모델의 정확도를 계산하는 함수 정의
def evaluate_model(model, texts, labels):
    predictions = model(texts)  # 모델 예측
    pred_labels = [int(pred["label"].split("_")[-1]) for pred in predictions]  # 예측 라벨 변환
    return np.mean(np.array(pred_labels) == np.array(labels))  # 정확도 계산

# 3번의 평가를 수행
results = {model_names[0]: [], model_names[1]: []}  # 결과를 저장할 딕셔너리 초기화

for _ in range(3):  # 3번 반복
    for model_name, model in zip(model_names, models):  # 각 모델에 대해 평가
        accuracy = evaluate_model(model, texts, labels)  # 정확도 계산
        results[model_name].append(accuracy)  # 결과 저장

# 각 모델의 평균 정확도 계산
average_results = {model: np.mean(accs) for model, accs in results.items()}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passe

In [13]:
# 가장 높은 평균 정확도를 가진 모델 찾기
best_model = max(average_results, key=average_results.get)
best_model, average_results[best_model]

('beomi/KcELECTRA-base', 0.53)

In [ ]:
!git add .

In [ ]:
!git commit -m "final exam"

In [ ]:
!git push origin main